### AD

In [ ]:
# aivietnam.ai
# simple stereo matching using pixel-wise matching

import cv2
import numpy as np

# cho trước chiều rộng và chiều cao của ảnh
height = 288
width  = 384

def stereo_matching(left_img, right_img, disparity_range):   
    # đọc ảnh trái và ảnh phải, rồi chuyển sang ảnh grayscale
    left  = cv2.imread(left_img, 0)    
    right = cv2.imread(right_img, 0)  
    
    # compute cost space
    cost_space = np.zeros((height, width, disparity_range))
    for j in range(disparity_range):                
        left_d  = left[:,j:width]
        right_d = right[:, 0:width-j]
        cost = np.abs(left_d-right_d)
        cost_space[:, j:width, j] = cost     
        
    # compute disparity from cost space
    depth = np.zeros((height, width), np.uint8)               
    scale = 255 / disparity_range
    
    for y in range(height):        
        for x in range(width):
            disparity = 0
            cost_min  = cost_space[y, x, 0]
            for j in range(1, disparity_range): 
                cost = cost_space[y, x, j]
                
                if cost < cost_min:
                    cost_min  = cost
                    disparity = j
                    
            # đã tìm được j (lưu ở biến disparity) để cost min
            # gán j đó vào disaprity map
            # nhân cho scale để nhìn thấy rõ ràng (không cần scale cũng được)
            depth[y, x] = disparity*scale
                
    # chuyển dữ liệu từ ndarray sang kiểu Image và lưu xuống file
    cv2.imwrite('images/disparity_ad_np.png', depth)  
         
if __name__ == '__main__':
    disparity_range = 16 # cho cặp ảnh Tsukuba
    stereo_matching("images/left.png", "images/right.png", disparity_range)

In [1]:
# aivietnam.ai
# simple stereo matching using pixel-wise matching

import cv2
import numpy as np

# cho trước chiều rộng và chiều cao của ảnh
height = 288
width  = 384

def stereo_matching(left_img, right_img, disparity_range):   
    # đọc ảnh trái và ảnh phải, rồi chuyển sang ảnh grayscale
    left  = cv2.imread(left_img, 0)    
    right = cv2.imread(right_img, 0)  
    
    # compute cost space
    cost_space = np.zeros((height, width, disparity_range))
    for j in range(disparity_range):                
        left_d  = left[:,j:width]
        right_d = right[:, 0:width-j]
        cost = np.abs(left_d-right_d)
        cost_space[:, j:width, j] = cost     
        
    # compute disparity from cost space           
    scale = 255 / disparity_range    
    depth = np.argmin(cost_space, axis=2)
    depth = depth*scale
    depth = depth.astype(np.uint8)
                
    # chuyển dữ liệu từ ndarray sang kiểu Image và lưu xuống file
    cv2.imwrite('images/disparity_ad_np_2.png', depth)  
         
if __name__ == '__main__':
    disparity_range = 16 # cho cặp ảnh Tsukuba
    stereo_matching("images/left.png", "images/right.png", disparity_range)

### SAD

In [2]:
# aivietnam.ai
# simple stereo matching using pixel-wise matching

import cv2
import numpy as np

# cho trước chiều rộng và chiều cao của ảnh
height = 288
width  = 384

def compute_integral(data):
    sum_axis_0 = np.cumsum(data, axis=0)
    sum_axis_1 = np.cumsum(sum_axis_0, axis=1)
    return sum_axis_1

def stereo_matching(left_img, right_img, d_range):   
    # đọc ảnh trái và ảnh phải, rồi chuyển sang ảnh grayscale
    left  = cv2.imread(left_img, 0)    
    right = cv2.imread(right_img, 0)  
    
    left  = left.astype(np.float32)
    right = right.astype(np.float32)
    
    # compute cost space
    cost_space = np.full((height, width, d_range), 255, np.float32)
    for j in range(disparity_range):                
        left_d  = left[:,j:width]
        right_d = right[:, 0:width-j]
        cost_d  = np.abs(left_d-right_d)
        cost_space[:, j:width, j] = cost_d
    
    # compute integral image space
    integral = np.full((height, width, d_range), 255, np.float32)
    for j in range(disparity_range):
        integral[:,:,j] = compute_integral(cost_space[:,:,j])
    
    # compute sum
    side = 1
    cost_sad_space = np.full((height, width, d_range), 255, np.float32)
    for i in range(side+1, height-side):        
        for j in range(side+1, width-side):
            for d in range(disparity_range):
                cost_sad_space[i, j, d] = integral[i+side,j+side, d] + integral[i-side-1,j-side-1, d] - integral[i-side-1,j+side, d] - integral[i+side,j-side-1, d]
   
    # compute disparity from cost space
    scale = 255 / disparity_range    
    depth = np.argmin(cost_sad_space, axis=2)
    depth = depth*scale
    depth = depth.astype(np.uint8)
                
    # chuyển dữ liệu từ ndarray sang kiểu Image và lưu xuống file
    cv2.imwrite('images/disparity_sad_np_2.png', depth)  
         
if __name__ == '__main__':
    disparity_range = 16 # cho cặp ảnh Tsukuba
    stereo_matching("images/left.png", "images/right.png", disparity_range)

In [6]:
# aivietnam.ai
# simple stereo matching using pixel-wise matching

import cv2
import numpy as np

# cho trước chiều rộng và chiều cao của ảnh
height = 288
width  = 384

def compute_integral_image(data):
    sum_axis_0 = np.cumsum(data, axis=0)
    sum_axis_1 = np.cumsum(sum_axis_0, axis=1)
    return sum_axis_1

def stereo_matching(left_img, right_img, disparity_range):   
    # đọc ảnh trái và ảnh phải, rồi chuyển sang ảnh grayscale
    left  = cv2.imread(left_img, 0)    
    right = cv2.imread(right_img, 0)  
    
    left  = left.astype(np.float32)
    right = right.astype(np.float32)
    
    # compute cost space
    cost_space = np.full((height, width, disparity_range), 255, np.float32)
    for j in range(disparity_range):                
        left_d  = left[:,j:width]
        right_d = right[:, 0:width-j]
        cost = np.abs(left_d-right_d)
        cost_space[:, j:width, j] = cost
    
    # compute integral image space
    integral = np.full((height, width, disparity_range), 255, np.float32)
    for j in range(disparity_range):
        integral[:,:,j] = compute_integral_image(cost_space[:,:,j])
    
    # compute sum
    side = 3
    cost_sad_space = np.full((height, width, disparity_range), 255, np.float32)
    
    cost_sad_space[:height-side, :width-side, :] = integral[side:, side:, :]
    cost_sad_space[side+1:, side+1:, :] = cost_sad_space[side+1:, side+1:, :] + integral[:height-side-1, :width-side-1, :]
    cost_sad_space[side+1:, :width-side, :] = cost_sad_space[side+1:, :width-side, :] - integral[:height-side-1, side:, :]
    cost_sad_space[:height-side, side+1:, :] = cost_sad_space[:height-side, side+1:, :] - integral[side:, :width-side-1, :]
       
    # compute disparity from cost space
    scale = 255 / disparity_range    
    depth = np.argmin(cost_sad_space, axis=2)
    depth = depth*scale
    depth = depth.astype(np.uint8)
                
    # chuyển dữ liệu từ ndarray sang kiểu Image và lưu xuống file
    cv2.imwrite('images/disparity_sad2.png', depth)  
         
if __name__ == '__main__':
    disparity_range = 16 # cho cặp ảnh Tsukuba
    stereo_matching("images/left.png", "images/right.png", disparity_range)